In [4]:
import multiprocessing
n_cpu = multiprocessing.cpu_count()
print(n_cpu)

12


In [11]:
import numpy as np

a = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
b = [[8, 7, 6], [5, 4, 3], [2, 1, 0]]
a = np.array(a)
b = np.array(b)
print(b*a)
print(np.ones((3,3))*1.1)

[[ 0  7 12]
 [15 16 15]
 [12  7  0]]
[[1.1 1.1 1.1]
 [1.1 1.1 1.1]
 [1.1 1.1 1.1]]


In [9]:
import numpy as np
import cv2

a = cv2.imread('raw_image.tif')
b = np.uint8(255)
print(type(a))
print(type(b))

<class 'numpy.ndarray'>
<class 'numpy.uint8'>


In [2]:
import numpy as np
i = range(10)
print(np.size(i))

10


In [ ]:
import numpy as np
i = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
print(i[-6:-1])

[]


In [61]:
import numpy as np
i = list(range(1,31))
print(i[10:20:-1])

[]


In [2]:
import numpy as np
a = [[1-1j,2-1j],[3-1j,4-1j]]
a = np.array(a)
print(a.real)

[[1. 2.]
 [3. 4.]]


In [ ]:
import psf
import math
import numpy as np
from numpy.fft import fft, ifft, fftshift, ifftshift

physical_pixel_size = 6.5/100
dx = physical_pixel_size
light_wavelength = 0.660
numerical_aperture = 1.3
abbe_diffraction_limit = light_wavelength/(2*numerical_aperture)
padding_size = 4*math.ceil(abbe_diffraction_limit/physical_pixel_size)
raw_image_size_x = 512
raw_image_size_y = 512

psf_type = "airy_disk"

grid_physical_1D_x = dx * np.arange(-(raw_image_size_x/2 + padding_size),(raw_image_size_x/2 + padding_size)) # in micrometers
grid_physical_1D_y = dx * np.arange(-(raw_image_size_y/2 + padding_size),(raw_image_size_y/2 + padding_size)) # in micrometers

grid_physical_length_x = (raw_image_size_x + 2*padding_size - 1)*dx			
grid_physical_length_y = (raw_image_size_y + 2*padding_size - 1)*dx

df_x = 1/(grid_physical_length_x) # Physcially correct spacing in spatial frequency space in units of micrometer^-1 
df_y = 1/(grid_physical_length_y) # Physcially correct spacing in spatial frequency space in units of micrometer^-1

f_corrected_grid_1D_x = df_x * np.arange(-(raw_image_size_x/2 + padding_size),(raw_image_size_x/2 + padding_size)) # in units of micrometer^-1
f_corrected_grid_1D_y = df_y * np.arange(-(raw_image_size_y/2 + padding_size),(raw_image_size_y/2 + padding_size)) # in units of micrometer^-1

mtf_on_grid = np.zeros((raw_image_size_x+2*padding_size, raw_image_size_y+2*padding_size))
psf_on_grid = np.zeros((raw_image_size_x+2*padding_size, raw_image_size_y+2*padding_size))

if psf_type == "airy_disk":
        
        for j in range(raw_image_size_y + 2*padding_size):
            for i in range(raw_image_size_y + 2*padding_size):
                x_e = [grid_physical_1D_x[i], grid_physical_1D_y[j]]
                psf_on_grid[i, j] =  psf.incoherent_PSF_airy_disk([0.0, 0.0], x_e, light_wavelength, numerical_aperture)
 		
        normalization = np.sum(psf_on_grid) * dx^2
        psf_on_grid = psf_on_grid / normalization
        intermediate_img = fftshift(fft(ifftshift(psf_on_grid)))

        for j in range(raw_image_size_y + 2*padding_size):
            for i in range(raw_image_size_x + 2*padding_size):
                mtf_on_grid[i, j] = psf.MTF_air_disk([f_corrected_grid_1D_x[i], f_corrected_grid_1D_y[j]], light_wavelength, numerical_aperture)
                if mtf_on_grid[i, j] == 0.0:
                    intermediate_img[i, j] = 0.0 * intermediate_img[i, j]

        FFT_point_spread_function = ifftshift(intermediate_img)


In [20]:
import psf

raw_image_size_x = 16
raw_image_size_y = 16
padding_size = 4
light_wavelength = 0.660
numerical_aperture = 1.3
grid_physical_1D_x = np.arange(-(raw_image_size_x/2 + padding_size),(raw_image_size_x/2 + padding_size)) # in micrometers
grid_physical_1D_y = np.arange(-(raw_image_size_y/2 + padding_size),(raw_image_size_y/2 + padding_size)) # in micrometers

for j in range(raw_image_size_y + 2*padding_size):
    for i in range(raw_image_size_y + 2*padding_size):
        x_e = [grid_physical_1D_x[i], grid_physical_1D_y[j]]
        psf_on_grid[i, j] =  psf.incoherent_PSF_airy_disk([0.0, 0.0], x_e, light_wavelength, numerical_aperture)

TypeError: ufunc 'bitwise_xor' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [14]:
a= 2**3

In [63]:
import numpy as np
i = nested_list = [[i * 9 + j + 1 for j in range(9)] for i in range(9)]
i = np.array(i)
j = [[1,2,3],[4,5,6],[7,8,9]]
j = np.array(j)
print(np.flip(j, axis=0))
print(i[::-1,::-1])

[[7 8 9]
 [4 5 6]
 [1 2 3]]
[[81 80 79 78 77 76 75 74 73]
 [72 71 70 69 68 67 66 65 64]
 [63 62 61 60 59 58 57 56 55]
 [54 53 52 51 50 49 48 47 46]
 [45 44 43 42 41 40 39 38 37]
 [36 35 34 33 32 31 30 29 28]
 [27 26 25 24 23 22 21 20 19]
 [18 17 16 15 14 13 12 11 10]
 [ 9  8  7  6  5  4  3  2  1]]


In [14]:
import multiprocessing
n_cpu = multiprocessing.cpu_count()
print(n_cpu)

20


In [17]:
import numpy as np
print(np.arange(-16,16))

[-16 -15 -14 -13 -12 -11 -10  -9  -8  -7  -6  -5  -4  -3  -2  -1   0   1
   2   3   4   5   6   7   8   9  10  11  12  13  14  15]
